In [1]:
!git clone https://github.com/ruzanovad/diploma.git

Cloning into 'diploma'...
remote: Enumerating objects: 440, done.
remote: Counting objects: 100% (440/440), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 440 (delta 244), reused 342 (delta 146), pack-reused 0 (from 0)
Receiving objects: 100% (440/440), 28.95 MiB | 30.91 MiB/s, done.
Resolving deltas: 100% (244/244), done.


In [3]:
%cd diploma/cnn-rnn

/kaggle/working/diploma/cnn-rnn


In [3]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import torch
from torch.utils.checkpoint import checkpoint
from torch.utils.data import DataLoader, Dataset
from torch import nn, Tensor
# import torchdata.datapipes as dp
import matplotlib.pyplot as plt
from model import Image2Latex, Text
from pathlib import Path
import torchvision
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence
import glob
from tqdm.notebook import tqdm
import time
from nltk.metrics import edit_distance
from typing import Tuple
import pytorch_lightning as pl

In [ ]:
data_path = Path('/kaggle/input/im2latex100k')
img_path = Path('/kaggle/input/im2latex100k/formula_images_processed/formula_images_processed')

In [10]:
%reload_ext tensorboard
# %tensorboard --logdir tb_logs/
%tensorboard --logdir=your_log_dir --bind_all

<IPython.core.display.Javascript object>

In [16]:
%run main.py --batch-size 256 --accumulate-batch 256 --train --log-step 1 \
--num-workers 2 \
--max-epochs 20 --lr 0.01 --model-name conv_bilstm_lstm_predict --enc-type resnet_encoder \
--enc-dim 256 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.1 \
--decode-type beamsearch --beam-width 5 --grad-clip 0 --dataset 100k \
--img-path /kaggle/input/im2latex100k/formula_images_processed/formula_images_processed \
--data-path /kaggle/input/im2latex100k \
--vocab_file data/vocab/latex_tokens.json \
--predict-img-path /kaggle/input/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


==========[Train]==========


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

NameError: name 'exit' is not defined

In [15]:
!nvidia-smi

Mon Mar 24 14:31:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P0             31W /   70W |     497MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
torch.cuda.empty_cache()

In [ ]:
%run main.py --batch-size 8 --accumulate-batch 64 --predict --log-step 1 \
--train-sample 0 --test-sample 0 --val-sample 0 --num-workers 2 \
--max-epochs 20 --lr 0.01 --model-name conv_bilstm_lstm_predict --enc-type resnet_encoder \
--enc-dim 256 --dec-dim 512 --emb-dim 80 --attn-dim 512 --dropout 0.1 \
--decode-type beamsearch --beam-width 5 --grad-clip 0 --dataset 100k \
--img-path data/im2latex100k/formula_images_processed/formula_images_processed \
--data-path data/im2latex100k --vocab_file data/vocab/latex_tokens.json \
--predict-img-path data/im2latex100k/formula_images_processed/formula_images_processed/100009e256.png